In [ ]:
# HNSW Status High - 高层新增边方法测试
from io_utils import read_fbin, read_ibin
import faiss
import numpy as np
import time

print(f"FAISS版本: {faiss.__version__}")

# 数据路径
file_path = "/root/code/vectordbindexing/Text2Image/base.1M.fbin"
query_path = "/root/code/vectordbindexing/Text2Image/query.public.100K.fbin"
ground_truth_path = "/root/code/vectordbindexing/Text2Image/groundtruth.public.100K.ibin"

# 读取数据集
print("\n=== 读取数据集 ===")
print("读取图像向量...")
data_vector = read_fbin(file_path)
print(f"图像向量: {data_vector.shape}, dtype: {data_vector.dtype}")

print("读取查询向量...")
query_vector = read_fbin(query_path)
print(f"查询向量: {query_vector.shape}, dtype: {query_vector.dtype}")

# 使用前100K数据进行测试
train_data_vector = data_vector[:100000]
print(f"训练数据: {train_data_vector.shape}")


1.11.0


reading image vector: ---
<class 'numpy.ndarray'>
2 (1000000, 200) float32 200000000


reading querys: ---
<class 'numpy.ndarray'>
2 (100000, 200) float32 20000000


In [2]:
import hnsw_cosine_status_high as hnsw_cosine
import simple_sim_hash
import importlib
importlib.reload(hnsw_cosine)

# M=64 比较合适，甚至更宽的宽度
# 这里是个经验值：会在增加宽度的同时，逐渐达到一个稳定值
index = hnsw_cosine.HNSWIndex(M=64, ef_construction=128, ef_search=64, random_seed=1)
simHash = simple_sim_hash.SimpleSimHash(dim=200)

IMAGE_IDX_SET = set()

# 形状 [N,200]（先用1M子集或更小切片做原型）
for img_id, vec in enumerate(train_data_vector):        # 可加 tqdm、批量 flush
    index.add_item_fast10k(vec, lsh=simHash, limit=100)
    IMAGE_IDX_SET.add(img_id)

for qid, vec in enumerate(query_vector):
    index.add_item_fast10k(vec, lsh=simHash, limit=100)

In [3]:
# 读取faiss搜索结果，获取 query_vector 和 search 结果
import json
train_query_list = {}
test_query_list = {}

with open("./TempResults/search_results_100K.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    for query_idx, vec_list in data.items():
        mList = []
        for x in vec_list:
            mList.append(x - int(query_idx))
        if int(query_idx) % 6 != 0:
            train_query_list[int(query_idx)] = mList
        else:
            test_query_list[int(query_idx)] = mList
print(f"num of train: {len(train_query_list)}")
print(f"num of test: {len(test_query_list)}")

num of train: 83333
num of test: 16667


In [4]:
# OOD search steps
NUM_STEPS = []
PHASE_ANALYSIS = []
for qid, target_list in test_query_list.items():
    q = query_vector[qid]
    for target_id in target_list[:10]:
        if target_id not in IMAGE_IDX_SET:
            continue
        # 使用阶段分析功能
        out = index.search_steps_to_target(q, target_id, k=10, ef=64, analyze_phases=True, verbose=False, multi_path_search=True, max_paths=3)
        NUM_STEPS.append(len(out["trace"]))
        if "phase_analysis" in out:
            PHASE_ANALYSIS.append(out["phase_analysis"])


# 分析阶段统计
if PHASE_ANALYSIS:
    print("\n=== 阶段分析统计 ===")
    phase_1_steps = [pa["phase_1"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_2_steps = [pa["phase_2"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_1_accel_edges = [pa["phase_1"]["accel_edges"] for pa in PHASE_ANALYSIS]
    phase_2_accel_edges = [pa["phase_2"]["accel_edges"] for pa in PHASE_ANALYSIS]
    
    print(f"第一阶段 (快速靠近) - 平均步数: {np.mean(phase_1_steps):.2f}, 平均加速边: {np.mean(phase_1_accel_edges):.2f}")
    print(f"第二阶段 (Beam Search) - 平均步数: {np.mean(phase_2_steps):.2f}, 平均加速边: {np.mean(phase_2_accel_edges):.2f}")
    
    # 计算加速边使用比例
    total_accel_edges = [pa["total_accel_edges"] for pa in PHASE_ANALYSIS]
    total_steps = [pa["total_steps"] for pa in PHASE_ANALYSIS]
    accel_edge_ratios = [accel/steps if steps > 0 else 0 for accel, steps in zip(total_accel_edges, total_steps)]
    
    print(f"整体加速边使用比例: {np.mean(accel_edge_ratios):.2%}")
    
    # 分析哪些查询受益最多
    if len(PHASE_ANALYSIS) > 0:
        best_benefit_idx = np.argmax(accel_edge_ratios)
        best_benefit = PHASE_ANALYSIS[best_benefit_idx]
        print(f"\n加速边受益最多的查询:")
        print(f"  第一阶段: {best_benefit['phase_1']['step_count']} 步, {best_benefit['phase_1']['accel_edges']} 条加速边")
        print(f"  第二阶段: {best_benefit['phase_2']['step_count']} 步, {best_benefit['phase_2']['accel_edges']} 条加速边")
        print(f"  总步数: {best_benefit['total_steps']}, 总加速边: {best_benefit['total_accel_edges']}")
        print(f"  加速边比例: {best_benefit['overall_accel_edge_ratio']:.2%}")


arr_ori_bak = np.array(NUM_STEPS, dtype=np.float64)
arr_ori = arr_ori_bak.copy()
arr_ori.sort()

mean_steps = arr_ori.mean()
P50_steps = np.percentile(arr_ori, 50)
p99_steps = np.percentile(arr_ori, 99)
print(f"\n原始搜索统计:")
print(f"mean steps: {mean_steps}")
print(f"middle steps: {P50_steps}")
print(f"p99 steps: {p99_steps}")


=== 阶段分析统计 ===
第一阶段 (快速靠近) - 平均步数: 406.25, 平均加速边: 0.00
第二阶段 (Beam Search) - 平均步数: 72.71, 平均加速边: 0.00
整体加速边使用比例: 0.00%

加速边受益最多的查询:
  第一阶段: 412 步, 0 条加速边
  第二阶段: 110 步, 0 条加速边
  总步数: 522, 总加速边: 0
  加速边比例: 0.00%

原始搜索统计:
mean steps: 478.96458644371455
middle steps: 460.0
p99 steps: 906.0


In [10]:
# 使用新的 RoarGraph 风格的 cross distribution 边构建
print("\n=== 构建 RoarGraph 风格的 Cross Distribution 边 ===")
for query in query_vector:
    stats = index.build_cross_distribution_edges(
        max_new_edges_per_node=4,
        query=query,
    )
print("Cross distribution 边构建统计:")
print(stats)

# 获取 cross distribution 边的统计信息
cross_stats = index.get_cross_distribution_stats()
print("\nCross distribution 边统计:")
print(f"总添加的 cross distribution 边: {cross_stats['total_cross_edges']}")
print(f"被删除的 cross distribution 边: {cross_stats['deleted_cross_edges']}")
print(f"活跃的 cross distribution 边: {cross_stats['active_cross_edges']}")



=== 构建 RoarGraph 风格的 Cross Distribution 边 ===


Cross distribution 边构建统计:
{'query_processed': True, 'layer_1_nodes_total': 3092, 'top_k_selected': 10, 'pairs_considered': 45, 'pairs_added': 10, 'skipped_existing': 35, 'pruned_by_cap': 0, 'edges_added': 10, 'top_k_nodes': [123656, 125677, 158436, 183136, 132312, 179175, 141094, 190128, 156261, 174419], 'query_distance': 0.3494441509246826}

Cross distribution 边统计:
总添加的 cross distribution 边: 1512998
被删除的 cross distribution 边: 1374116
活跃的 cross distribution 边: 138882


In [11]:
# OOD search steps
NUM_STEPS = []
PHASE_ANALYSIS = []
for qid, target_list in test_query_list.items():
    q = query_vector[qid]
    for target_id in target_list[:10]:
        if target_id not in IMAGE_IDX_SET:
            continue
        # 使用阶段分析功能
        out = index.search_steps_to_target(q, target_id, k=10, ef=64, analyze_phases=True, verbose=False, multi_path_search=True, max_paths=3)
        NUM_STEPS.append(len(out["trace"]))
        if "phase_analysis" in out:
            PHASE_ANALYSIS.append(out["phase_analysis"])


# 分析阶段统计
if PHASE_ANALYSIS:
    print("\n=== 阶段分析统计 ===")
    phase_1_steps = [pa["phase_1"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_2_steps = [pa["phase_2"]["step_count"] for pa in PHASE_ANALYSIS]
    phase_1_accel_edges = [pa["phase_1"]["accel_edges"] for pa in PHASE_ANALYSIS]
    phase_2_accel_edges = [pa["phase_2"]["accel_edges"] for pa in PHASE_ANALYSIS]
    
    print(f"第一阶段 (快速靠近) - 平均步数: {np.mean(phase_1_steps):.2f}, 平均加速边: {np.mean(phase_1_accel_edges):.2f}")
    print(f"第二阶段 (Beam Search) - 平均步数: {np.mean(phase_2_steps):.2f}, 平均加速边: {np.mean(phase_2_accel_edges):.2f}")
    
    # 计算加速边使用比例
    total_accel_edges = [pa["total_accel_edges"] for pa in PHASE_ANALYSIS]
    total_steps = [pa["total_steps"] for pa in PHASE_ANALYSIS]
    accel_edge_ratios = [accel/steps if steps > 0 else 0 for accel, steps in zip(total_accel_edges, total_steps)]
    
    print(f"整体加速边使用比例: {np.mean(accel_edge_ratios):.2%}")
    
    # 分析哪些查询受益最多
    if len(PHASE_ANALYSIS) > 0:
        best_benefit_idx = np.argmax(accel_edge_ratios)
        best_benefit = PHASE_ANALYSIS[best_benefit_idx]
        print(f"\n加速边受益最多的查询:")
        print(f"  第一阶段: {best_benefit['phase_1']['step_count']} 步, {best_benefit['phase_1']['accel_edges']} 条加速边")
        print(f"  第二阶段: {best_benefit['phase_2']['step_count']} 步, {best_benefit['phase_2']['accel_edges']} 条加速边")
        print(f"  总步数: {best_benefit['total_steps']}, 总加速边: {best_benefit['total_accel_edges']}")
        print(f"  加速边比例: {best_benefit['overall_accel_edge_ratio']:.2%}")


arr_ori_bak = np.array(NUM_STEPS, dtype=np.float64)
arr_ori = arr_ori_bak.copy()
arr_ori.sort()

mean_steps = arr_ori.mean()
P50_steps = np.percentile(arr_ori, 50)
p99_steps = np.percentile(arr_ori, 99)
print(f"\n原始搜索统计:")
print(f"mean steps: {mean_steps}")
print(f"middle steps: {P50_steps}")
print(f"p99 steps: {p99_steps}")


=== 阶段分析统计 ===
第一阶段 (快速靠近) - 平均步数: 525.21, 平均加速边: 275.88
第二阶段 (Beam Search) - 平均步数: 72.71, 平均加速边: 0.00
整体加速边使用比例: 47.27%

加速边受益最多的查询:
  第一阶段: 372 步, 267 条加速边
  第二阶段: 0 步, 0 条加速边
  总步数: 372, 总加速边: 267
  加速边比例: 71.77%

原始搜索统计:
mean steps: 597.9242759032547
middle steps: 585.0
p99 steps: 1040.0


In [ ]:
# 新增：真实Recall测试功能
print("\n=== 新增：真实Recall@100测试 ===")

# 读取ground truth数据
print("读取ground truth数据...")
ground_truth = read_ibin(ground_truth_path)
print(f"Ground truth形状: {ground_truth.shape}")

def calculate_recall_at_k(predicted_ids, ground_truth_ids, k):
    """
    计算recall@k
    
    Args:
        predicted_ids: 预测的top-k结果
        ground_truth_ids: ground truth结果
        k: top-k值
    
    Returns:
        recall@k值
    """
    # 取前k个预测结果
    top_k_pred = set(predicted_ids[:k])
    
    # 取ground truth中在索引范围内的结果
    valid_gt = set()
    for gt_id in ground_truth_ids:
        if gt_id in IMAGE_IDX_SET:  # 确保ground truth在索引中
            valid_gt.add(gt_id)
    
    if len(valid_gt) == 0:
        return 0.0
    
    # 计算交集
    intersection = top_k_pred.intersection(valid_gt)
    
    # recall@k = |intersection| / |ground_truth|
    recall = len(intersection) / len(valid_gt)
    return recall

# 验证ground truth数据格式
print(f"\n=== Ground Truth数据格式分析 ===")
print(f"前5个查询的ground truth:")
for i in range(min(5, len(ground_truth))):
    print(f"  查询{i}: {ground_truth[i][:10]}... (共{len(ground_truth[i])}个)")

# 验证ground truth索引是否在范围内
gt_min, gt_max = ground_truth.min(), ground_truth.max()
print(f"\nGround truth索引范围: {gt_min} - {gt_max}")
print(f"训练数据索引范围: 0 - {len(IMAGE_IDX_SET)-1}")
valid_gt_count = np.sum(np.isin(ground_truth, list(IMAGE_IDX_SET)))
total_gt_count = ground_truth.size
print(f"Ground truth中有多少在训练数据范围内: {valid_gt_count} / {total_gt_count} ({valid_gt_count/total_gt_count*100:.1f}%)")


In [ ]:
# 测试真实的recall - 返回top100并与ground truth对比
print("\n=== 测试1: 真实Recall@100测试 ===")

# 选择测试查询（使用前500个查询进行测试）
test_query_count = 500
test_queries = query_vector[:test_query_count]
test_ground_truth = ground_truth[:test_query_count]

# 不同ef_search值的测试
ef_values = [32, 64, 128, 256]
recall_results = {}
search_time_results = {}
search_steps_results = {}

for ef in ef_values:
    print(f"\n测试 ef_search={ef}...")
    
    recalls = []
    search_times = []
    search_steps_list = []
    
    for i, (query, gt) in enumerate(zip(test_queries, test_ground_truth)):
        # 搜索
        start_time = time.time()
        results, search_steps = index.query_with_steps(query, k=100, ef=ef)
        search_time = time.time() - start_time
        
        # 计算recall@100
        recall = calculate_recall_at_k(results, gt, 100)
        
        recalls.append(recall)
        search_times.append(search_time)
        search_steps_list.append(search_steps)
        
        if (i + 1) % 50 == 0:
            print(f"  完成 {i+1}/{test_query_count} 个查询")
    
    # 统计结果
    recall_results[ef] = {
        'mean': np.mean(recalls),
        'std': np.std(recalls),
        'min': np.min(recalls),
        'max': np.max(recalls),
        'p50': np.percentile(recalls, 50),
        'p95': np.percentile(recalls, 95),
        'p99': np.percentile(recalls, 99)
    }
    
    search_time_results[ef] = {
        'mean': np.mean(search_times),
        'std': np.std(search_times),
        'min': np.min(search_times),
        'max': np.max(search_times)
    }
    
    search_steps_results[ef] = {
        'mean': np.mean(search_steps_list),
        'std': np.std(search_steps_list),
        'min': np.min(search_steps_list),
        'max': np.max(search_steps_list)
    }
    
    print(f"  ef_search={ef}: 平均recall@100={recall_results[ef]['mean']:.3f}")
    print(f"    平均搜索时间={search_time_results[ef]['mean']:.4f}s")
    print(f"    平均搜索步数={search_steps_results[ef]['mean']:.1f}")

# 显示recall结果汇总
print(f"\n=== Recall@100结果汇总 ===")
print(f"{'ef_search':<10} {'mean':<8} {'std':<8} {'p50':<8} {'p95':<8} {'p99':<8}")
print("-" * 60)
for ef in ef_values:
    r = recall_results[ef]
    print(f"{ef:<10} {r['mean']:<8.3f} {r['std']:<8.3f} {r['p50']:<8.3f} {r['p95']:<8.3f} {r['p99']:<8.3f}")

# 显示搜索时间汇总
print(f"\n=== 搜索时间结果汇总 (秒) ===")
print(f"{'ef_search':<10} {'mean':<10} {'std':<10} {'min':<10} {'max':<10}")
print("-" * 60)
for ef in ef_values:
    t = search_time_results[ef]
    print(f"{ef:<10} {t['mean']:<10.4f} {t['std']:<10.4f} {t['min']:<10.4f} {t['max']:<10.4f}")

# 显示搜索步数汇总
print(f"\n=== 搜索步数结果汇总 ===")
print(f"{'ef_search':<10} {'mean':<10} {'std':<10} {'min':<10} {'max':<10}")
print("-" * 60)
for ef in ef_values:
    s = search_steps_results[ef]
    print(f"{ef:<10} {s['mean']:<10.1f} {s['std']:<10.1f} {s['min']:<10.1f} {s['max']:<10.1f}")


In [ ]:
# 测试不同recall概率下的搜索步长
print("\n=== 测试2: 不同Recall概率下的搜索步长测试 ===")

def find_ef_for_recall_target(index, queries, ground_truths, target_recall, k=100, ef_range=(32, 512)):
    """
    找到达到目标recall所需的最小ef_search值
    
    Args:
        index: HNSW索引
        queries: 查询向量列表
        ground_truths: 对应的ground truth列表
        target_recall: 目标recall值 (如0.90, 0.95)
        k: top-k值
        ef_range: ef_search搜索范围 (min, max)
    
    Returns:
        (optimal_ef, achieved_recall, search_steps): 最优ef值、达到的recall、搜索步数
    """
    ef_min, ef_max = ef_range
    
    # 测试不同的ef值
    test_efs = [ef_min, ef_min + (ef_max - ef_min) // 4, ef_min + (ef_max - ef_min) // 2, 
                ef_min + 3 * (ef_max - ef_min) // 4, ef_max]
    
    best_ef = ef_max
    best_recall = 0.0
    
    for ef in test_efs:
        recalls = []
        search_steps = []
        
        for query, gt in zip(queries, ground_truths):
            results, steps = index.query_with_steps(query, k=k, ef=ef)
            recall = calculate_recall_at_k(results, gt, k)
            recalls.append(recall)
            search_steps.append(steps)
        
        mean_recall = np.mean(recalls)
        mean_steps = np.mean(search_steps)
        
        if mean_recall >= target_recall and ef < best_ef:
            best_ef = ef
            best_recall = mean_recall
    
    # 如果最小值就能达到目标，测试更小的值
    if mean_recall >= target_recall and ef == ef_min:
        for ef in [16, 24, 32]:
            recalls = []
            search_steps = []
            
            for query, gt in zip(queries, ground_truths):
                results, steps = index.query_with_steps(query, k=k, ef=ef)
                recall = calculate_recall_at_k(results, gt, k)
                recalls.append(recall)
                search_steps.append(steps)
            
            mean_recall = np.mean(recalls)
            mean_steps = np.mean(search_steps)
            
            if mean_recall >= target_recall and ef < best_ef:
                best_ef = ef
                best_recall = mean_recall
            else:
                break
    
    # 计算最优ef对应的搜索步数
    final_recalls = []
    final_steps = []
    for query, gt in zip(queries, ground_truths):
        results, steps = index.query_with_steps(query, k=k, ef=best_ef)
        recall = calculate_recall_at_k(results, gt, k)
        final_recalls.append(recall)
        final_steps.append(steps)
    
    return best_ef, np.mean(final_recalls), np.mean(final_steps)

# 测试不同recall目标（使用更小的查询集）
recall_targets = [0.70, 0.80, 0.85, 0.90]
recall_step_results = {}

# 使用100个查询进行快速测试
test_query_small = test_queries[:100]
test_gt_small = test_ground_truth[:100]

print("测试不同recall目标下的搜索步长...")
for target_recall in recall_targets:
    print(f"\n测试目标recall={target_recall*100:.0f}%...")
    
    optimal_ef, achieved_recall, search_steps = find_ef_for_recall_target(
        index, test_query_small, test_gt_small, target_recall, k=100
    )
    
    recall_step_results[target_recall] = {
        'ef_search': optimal_ef,
        'achieved_recall': achieved_recall,
        'search_steps': search_steps
    }
    
    print(f"  目标recall: {target_recall*100:.0f}%")
    print(f"  最优ef_search: {optimal_ef}")
    print(f"  达到的recall: {achieved_recall:.3f}")
    print(f"  平均搜索步数: {search_steps:.1f}")

# 显示结果汇总
print(f"\n=== 不同Recall目标下的搜索步长汇总 ===")
print(f"{'目标Recall':<12} {'最优ef':<10} {'达到Recall':<12} {'搜索步数':<10}")
print("-" * 50)
for target, results in recall_step_results.items():
    print(f"{target*100:>8.0f}%{'':<4} {results['ef_search']:<10} {results['achieved_recall']:<12.3f} {results['search_steps']:<10.1f}")


In [ ]:
# 可视化结果和总结报告
print("\n=== 测试3: 结果可视化 ===")

import matplotlib.pyplot as plt

# 绘制recall vs ef_search曲线
plt.figure(figsize=(15, 5))

# 子图1: Recall@100 vs ef_search
plt.subplot(1, 3, 1)
ef_list = list(recall_results.keys())
mean_recalls = [recall_results[ef]['mean'] for ef in ef_list]
std_recalls = [recall_results[ef]['std'] for ef in ef_list]

plt.errorbar(ef_list, mean_recalls, yerr=std_recalls, marker='o', capsize=5)
plt.xlabel('ef_search')
plt.ylabel('Recall@100')
plt.title('Recall@100 vs ef_search')
plt.grid(True, alpha=0.3)

# 子图2: 搜索时间 vs ef_search
plt.subplot(1, 3, 2)
mean_times = [search_time_results[ef]['mean'] for ef in ef_list]
std_times = [search_time_results[ef]['std'] for ef in ef_list]

plt.errorbar(ef_list, mean_times, yerr=std_times, marker='s', capsize=5, color='orange')
plt.xlabel('ef_search')
plt.ylabel('搜索时间 (秒)')
plt.title('搜索时间 vs ef_search')
plt.grid(True, alpha=0.3)

# 子图3: Recall目标 vs 搜索步数
plt.subplot(1, 3, 3)
target_list = list(recall_step_results.keys())
step_list = [recall_step_results[target]['search_steps'] for target in target_list]
ef_list_target = [recall_step_results[target]['ef_search'] for target in target_list]

plt.plot([t*100 for t in target_list], step_list, marker='^', linewidth=2, markersize=8)
plt.xlabel('目标Recall (%)')
plt.ylabel('搜索步数')
plt.title('不同Recall目标下的搜索步数')
plt.grid(True, alpha=0.3)

# 在第三个图上添加ef_search值作为标注
for i, (target, steps, ef) in enumerate(zip(target_list, step_list, ef_list_target)):
    plt.annotate(f'ef={ef}', (target*100, steps), 
                xytext=(5, 5), textcoords='offset points', fontsize=8)

plt.tight_layout()
plt.show()

# 总结报告
print("\n=== Recall测试总结报告 ===")
print(f"1. 真实Recall@100测试:")
print(f"   - 测试了 {test_query_count} 个查询")
print(f"   - ef_search范围: {ef_values}")
print(f"   - 最佳recall: {max(mean_recalls):.3f} (ef_search={ef_list[np.argmax(mean_recalls)]})")
print(f"   - 最快搜索: {min(mean_times):.4f}s (ef_search={ef_list[np.argmin(mean_times)]})")

print(f"\n2. 不同Recall目标下的搜索步长:")
print(f"   - 测试了 {len(recall_targets)} 个recall目标")
print(f"   - 最高目标recall: {max(recall_step_results.keys())*100:.0f}%")
if recall_step_results:
    print(f"   - 最大搜索步数: {max(step_list):.1f} (recall={max(recall_step_results.keys())*100:.0f}%)")
    print(f"   - 最小搜索步数: {min(step_list):.1f} (recall={min(recall_step_results.keys())*100:.0f}%)")

print(f"\n3. 性能建议:")
best_recall_ef = ef_list[np.argmax(mean_recalls)]
best_time_ef = ef_list[np.argmin(mean_times)]
print(f"   - 追求最高recall: 使用ef_search={best_recall_ef}")
print(f"   - 追求最快速度: 使用ef_search={best_time_ef}")
print(f"   - 平衡选择: ef_search=64-128 之间")

print(f"\n4. 数据质量分析:")
print(f"   - Ground truth覆盖率: {valid_gt_count}/{total_gt_count} ({valid_gt_count/total_gt_count*100:.1f}%)")
print(f"   - 这解释了为什么recall值可能较低")

print(f"\n5. 与原有搜索步数测试的对比:")
print(f"   - 原有测试主要关注搜索到特定目标的步数")
print(f"   - 新增测试关注整体recall性能和搜索效率")
print(f"   - 两者结合可以全面评估HNSW索引的性能")
